In [1]:
import tensorflow as tf
import numpy as np
import cv2
import keras_ocr

import os
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
import re
import six
import math
import lmdb

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

%matplotlib

print(tf.__version__)
print(np.__version__)
print(cv2.__version__)
print(keras_ocr.__version__)

Using matplotlib backend: agg
2.6.0
1.21.4
8.3.2
4.5.3
0.8.8


In [ ]:
BATCH_SIZE = 128

HOME_DIR = os.getenv('HOME')+'/aiffel/ocr'
# TRAIN_DATA_PATH = HOME_DIR+'/data/MJ/MJ_train'
# VALID_DATA_PATH = HOME_DIR+'/data/MJ/MJ_valid'
# TEST_DATA_PATH = HOME_DIR+'/data/MJ/MJ_test'